#### Import Packages

In [1]:
import pandas as pd
import numpy as np
import glob
pd.set_option("display.max_columns",None)
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

In [2]:
train_data = pd.read_excel("train.xlsx")
test_data = pd.read_excel("test.xlsx")

In [3]:
train_data.drop(["Unnamed: 0"],axis=1,inplace=True)
test_data.drop(["Unnamed: 0"],axis=1,inplace=True)

#### Train Data

In [2]:
train_data = pd.DataFrame()
for file in glob.glob("dataset\\train\\*.txt"):
    file1 = pd.read_fwf(file,header=None)
    file1 = np.transpose(file1)
    file1.columns=file1.iloc[0]
    file1 = file1.drop(index=(0))
    file1=file1[["ID", "Delivery_person_ID", "Delivery_person_Age",
       "Delivery_person_Ratings", "Restaurant_latitude",
       "Restaurant_longitude", "Delivery_location_latitude",
       "Delivery_location_longitude", "Order_Date", "Time_Orderd",
       "Time_Order_picked", "Weather conditions", "Road_traffic_density",
       "Vehicle_condition", "Type_of_order", "Type_of_vehicle",
       "multiple_deliveries", "Festival", "City", "Time_taken (min)"]]
    train_data = train_data.append(file1)

#### Test Data

In [3]:
test_data = pd.DataFrame()
for file in glob.glob("dataset\\test\\*.txt"):
    file1 = pd.read_fwf(file,header=None)
    file1 = np.transpose(file1)
    file1.columns=file1.iloc[0]
    file1 = file1.drop(index=(0))
    file1=file1[["ID", "Delivery_person_ID", "Delivery_person_Age",
       "Delivery_person_Ratings", "Restaurant_latitude",
       "Restaurant_longitude", "Delivery_location_latitude",
       "Delivery_location_longitude", "Order_Date", "Time_Orderd",
       "Time_Order_picked", "Weather conditions", "Road_traffic_density",
       "Vehicle_condition", "Type_of_order", "Type_of_vehicle",
       "multiple_deliveries", "Festival", "City"]]
    test_data = test_data.append(file1)

#### Data Preprocessing

In [4]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45593 entries, 0 to 45592
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ID                           45593 non-null  object 
 1   Delivery_person_ID           45593 non-null  object 
 2   Delivery_person_Age          43739 non-null  float64
 3   Delivery_person_Ratings      43685 non-null  float64
 4   Restaurant_latitude          45593 non-null  float64
 5   Restaurant_longitude         45593 non-null  float64
 6   Delivery_location_latitude   45593 non-null  float64
 7   Delivery_location_longitude  45593 non-null  float64
 8   Order_Date                   45593 non-null  object 
 9   Time_Orderd                  43862 non-null  object 
 10  Time_Order_picked            45593 non-null  object 
 11  Weather conditions           44977 non-null  object 
 12  Road_traffic_density         44992 non-null  object 
 13  Vehicle_conditio

In [5]:
train_data['City'] = train_data['City'].fillna(train_data['City'].mode()[0])
train_data['Weather conditions'] = train_data['Weather conditions'].fillna(train_data['Weather conditions'].mode()[0])

In [6]:
def NanValue(train_data,listdata, value, valtype):
    listdelvper = listdata.copy()
    listdelvper.append(value)
    train_data2=train_data[listdelvper]
    if valtype == "mean":
        train_data2=train_data2.dropna().groupby(listdata).mean().reset_index()
    else:
        train_data2["count"]=1
        train_data2=train_data2.dropna().groupby(listdelvper).count().reset_index()
        train_data2.sort_values(by=["count"],ascending=False,inplace=True)
        train_data2.drop(["count"],axis=1,inplace=True)
        train_data2.drop_duplicates(subset=listdata,keep='first',inplace=True)
    train_data2.rename({value:value+"notna"},axis=1,inplace=True)
    train_data=pd.merge(train_data,train_data2,how="left",on=listdata)
    train_data[value]=train_data[value].fillna(train_data[value+"notna"])
    train_data.drop([value+"notna"],axis=1,inplace=True)
    return train_data

In [7]:
train_data=NanValue(train_data,["Delivery_person_ID"],"Delivery_person_Age",'mean')
train_data=NanValue(train_data,["Delivery_person_ID"],"Delivery_person_Ratings",'mean')

train_data['Delivery_person_Age'].fillna(value=train_data['Delivery_person_Age'].mean(), inplace=True)
train_data['Delivery_person_Ratings'].fillna(value=train_data['Delivery_person_Ratings'].mean(), inplace=True)

In [8]:
import datetime
from datetime import datetime, timedelta

In [9]:
train_data["Order_Date"] = train_data["Order_Date"].apply(lambda x: datetime.strptime(x,"%d-%m-%Y").strftime("%Y-%m-%d"))
train_data["Order_Date"]=pd.to_datetime(train_data["Order_Date"], format ="%Y-%m-%d",errors ="coerce")

In [10]:
train_data["Time_Order_picked"] = np.where(train_data["Time_Order_picked"].str.contains(':60'),
                                           pd.to_datetime(train_data["Time_Order_picked"].str.replace(':60',':00'), format ="%H:%M",errors ="coerce")+timedelta(hours=1),
                                           np.where(train_data["Time_Order_picked"].str.contains('24:'),
                                                    pd.to_datetime(train_data["Time_Order_picked"].str.replace('24:','00:'), format ="%H:%M",errors ="coerce"),
                                                    pd.to_datetime(train_data["Time_Order_picked"], format ="%H:%M",errors ="coerce")))

In [11]:
train_data["Time_Orderd"] = np.where(train_data["Time_Orderd"].str.contains(':60'),
                                           pd.to_datetime(train_data["Time_Orderd"].str.replace(':60',':00'), format ="%H:%M",errors ="coerce")+timedelta(hours=1),
                                           np.where(train_data["Time_Orderd"].str.contains('24:'),
                                                    pd.to_datetime(train_data["Time_Orderd"].str.replace('24:','00:'), format ="%H:%M",errors ="coerce"),
                                                    pd.to_datetime(train_data["Time_Orderd"], format ="%H:%M",errors ="coerce")))

In [12]:
train_data["timediff"]=train_data["Time_Order_picked"]-train_data["Time_Orderd"]

In [13]:
train_data=NanValue(train_data,["Road_traffic_density","Weather conditions","City"],"timediff","mode")

C:\Users\alish\AppData\Local\Temp\ipykernel_132\1843462781.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [14]:
train_data['timediff'] = train_data['timediff'].fillna(train_data['timediff'].mode()[0])

In [15]:
train_data["Time_Orderd"]=train_data["Time_Orderd"].fillna(train_data["Time_Order_picked"]-train_data['timediff'])

In [16]:
train_data=NanValue(train_data,["Time_taken (min)","City"],"Road_traffic_density","mode")
train_data=NanValue(train_data,["Road_traffic_density","City","Order_Date"],"Festival","mode")
train_data=NanValue(train_data,["Order_Date","Time_Order_picked"],"multiple_deliveries","mode")

C:\Users\alish\AppData\Local\Temp\ipykernel_132\1843462781.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [17]:
train_data=NanValue(train_data,["Delivery_person_ID"],"multiple_deliveries","mode")
train_data['multiple_deliveries'] = train_data['multiple_deliveries'].fillna(train_data['multiple_deliveries'].mode()[0])

C:\Users\alish\AppData\Local\Temp\ipykernel_132\1843462781.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [18]:
train_data["Order_Year"]=train_data["Order_Date"].dt.year
train_data["Order_Month"]=train_data["Order_Date"].dt.month
train_data["Order_Day"]=train_data["Order_Date"].dt.day

In [19]:
train_data["Time_Orderd_hr"]=train_data["Time_Orderd"].dt.hour
train_data["Time_Orderd_minute"]=train_data["Time_Orderd"].dt.minute

train_data["Time_Order_picked_hr"]=train_data["Time_Order_picked"].dt.hour
train_data["Time_Order_picked_minute"]=train_data["Time_Order_picked"].dt.minute

In [20]:
train_data.drop(["Order_Date","Time_Orderd","Time_Order_picked"],axis=1,inplace=True)

In [21]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45593 entries, 0 to 45592
Data columns (total 25 columns):
 #   Column                       Non-Null Count  Dtype          
---  ------                       --------------  -----          
 0   ID                           45593 non-null  object         
 1   Delivery_person_ID           45593 non-null  object         
 2   Delivery_person_Age          45593 non-null  float64        
 3   Delivery_person_Ratings      45593 non-null  float64        
 4   Restaurant_latitude          45593 non-null  float64        
 5   Restaurant_longitude         45593 non-null  float64        
 6   Delivery_location_latitude   45593 non-null  float64        
 7   Delivery_location_longitude  45593 non-null  float64        
 8   Weather conditions           45593 non-null  object         
 9   Road_traffic_density         45593 non-null  object         
 10  Vehicle_condition            45593 non-null  int64          
 11  Type_of_order               

#### Finding Correlation

In [22]:
numerical = ['Delivery_person_Ratings', 'Restaurant_latitude', 'Restaurant_longitude', 'Delivery_location_latitude', 
             'Delivery_location_longitude','Vehicle_condition', 'multiple_deliveries','Order_Year', 'Order_Month', 'Order_Day', 
             'Time_Orderd_hr', 'Time_Orderd_minute', 'Time_Order_picked_hr', 'Time_Order_picked_minute']
data = [ go.Heatmap(
        z= train_data[numerical].astype(float).corr().values, # Pearson correlation
        x=train_data[numerical].columns.values,
        y=train_data[numerical].columns.values,
        colorscale='Viridis',
        reversescale = False,
#         text = True ,
        opacity = 1.0
        
    )
]


layout = go.Layout(
    title='Pearson Correlation of numerical features',
    xaxis = dict(ticks='', nticks=36),
    yaxis = dict(ticks='' ),
    width = 900, height = 700,
    
)


fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='labelled-heatmap')

#### Catergorical Data Encoding

In [23]:
train_data_base = train_data.copy()

In [24]:
def categoricalDataEndcoding(train_data,train_data_base,catcol):
    WC=list(train_data_base[catcol].unique())
    WC_dict={k: v for v, k in enumerate(WC)}
    train_data[catcol] = train_data[catcol].apply(lambda x: WC_dict[x])
    return train_data

In [25]:
catergoryData = ["Weather conditions","Road_traffic_density","Type_of_order","Type_of_vehicle","Festival","City"]
for i in catergoryData:
    train_data = categoricalDataEndcoding(train_data,train_data_base,i)

#### Splitting the dataset into the Training set and Test set

In [26]:
train_data.columns

Index(['ID', 'Delivery_person_ID', 'Delivery_person_Age',
       'Delivery_person_Ratings', 'Restaurant_latitude',
       'Restaurant_longitude', 'Delivery_location_latitude',
       'Delivery_location_longitude', 'Weather conditions',
       'Road_traffic_density', 'Vehicle_condition', 'Type_of_order',
       'Type_of_vehicle', 'multiple_deliveries', 'Festival', 'City',
       'Time_taken (min)', 'timediff', 'Order_Year', 'Order_Month',
       'Order_Day', 'Time_Orderd_hr', 'Time_Orderd_minute',
       'Time_Order_picked_hr', 'Time_Order_picked_minute'],
      dtype='object')

In [27]:
X = train_data[['Delivery_person_Age',
       'Delivery_person_Ratings', 'Restaurant_latitude',
       'Restaurant_longitude', 'Delivery_location_latitude',
       'Delivery_location_longitude', 'Weather conditions',
       'Road_traffic_density', 'Vehicle_condition', 'Type_of_order',
       'Type_of_vehicle', 'multiple_deliveries', 'Festival', 'City','Order_Month',
       'Order_Day', 'Time_Orderd_hr', 'Time_Orderd_minute',
       'Time_Order_picked_hr', 'Time_Order_picked_minute']].values
Y = train_data[["Time_taken (min)"]].values

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 1)

#### Feature Scaling

In [29]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [30]:
X_train

array([[-0.44842796,  0.81196974,  0.18493398, ...,  1.5304376 ,
        -0.95946917, -1.62525006],
       [ 1.4818445 , -0.41318623,  0.69428984, ...,  1.22975631,
         0.53683199, -1.34062528],
       [-1.32582453, -1.6383422 ,  0.05233792, ..., -1.77705661,
        -0.95946917, -1.05600051],
       ...,
       [-1.67678316,  0.50568075,  0.04890657, ..., -1.77705661,
         0.1627567 , -1.05600051],
       [-0.9748659 , -1.6383422 , -2.08378849, ...,  0.32771244,
        -1.33354445,  0.65174812],
       [ 0.77992724,  0.19939176,  0.69802831, ...,  0.02703114,
        -0.02428095,  0.65174812]])

#### Model Selection

In [31]:
# from sklearn.linear_model import LinearRegression
# regressor = LinearRegression()
# regressor.fit(X_train, y_train)

# from sklearn.svm import SVR
# regressor = SVR(kernel = 'rbf')
# regressor.fit(X_train, y_train)

from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
regressor.fit(X_train, y_train)

C:\Users\alish\AppData\Local\Temp\ipykernel_132\2449047362.py:11: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



RandomForestRegressor(n_estimators=10, random_state=0)

In [32]:
y_pred = regressor.predict(X_test)

In [33]:
df_preds = pd.DataFrame({'Actual': y_test.squeeze(), 'Predicted': y_pred.squeeze()})
print(df_preds)

      Actual  Predicted
0       23.0       23.7
1       25.0       19.7
2       32.0       33.0
3       36.0       37.0
4       17.0       14.0
...      ...        ...
9114    30.0       28.3
9115    20.0       21.7
9116    15.0       23.9
9117    24.0       27.1
9118    24.0       22.9

[9119 rows x 2 columns]


In [34]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f'Mean absolute error: {mae:.2f}')
print(f'Mean squared error: {mse:.2f}')
print(f'Root mean squared error: {rmse:.2f}')

Mean absolute error: 3.76
Mean squared error: 23.49
Root mean squared error: 4.85


In [35]:
Score = 100-mse
print(Score)

76.50663449939687


### Test Data

In [36]:
test_data['City'] = test_data['City'].fillna(test_data['City'].mode()[0])
test_data['Weather conditions'] = test_data['Weather conditions'].fillna(test_data['Weather conditions'].mode()[0])

test_data=NanValue(test_data,["Delivery_person_ID"],"Delivery_person_Age",'mean')
test_data=NanValue(test_data,["Delivery_person_ID"],"Delivery_person_Ratings",'mean')


test_data['Delivery_person_Age'].fillna(value=test_data['Delivery_person_Age'].mean(), inplace=True)
test_data['Delivery_person_Ratings'].fillna(value=test_data['Delivery_person_Ratings'].mean(), inplace=True)

test_data["Order_Date"] = test_data["Order_Date"].apply(lambda x: datetime.strptime(x,"%d-%m-%Y").strftime("%Y-%m-%d"))
test_data["Order_Date"]=pd.to_datetime(test_data["Order_Date"], format ="%Y-%m-%d",errors ="coerce")

test_data["Time_Order_picked"] = np.where(test_data["Time_Order_picked"].str.contains(':60'),
                                           pd.to_datetime(test_data["Time_Order_picked"].str.replace(':60',':00'), format ="%H:%M",errors ="coerce")+timedelta(hours=1),
                                           np.where(test_data["Time_Order_picked"].str.contains('24:'),
                                                    pd.to_datetime(test_data["Time_Order_picked"].str.replace('24:','00:'), format ="%H:%M",errors ="coerce"),
                                                    pd.to_datetime(test_data["Time_Order_picked"], format ="%H:%M",errors ="coerce")))

test_data["Time_Orderd"] = np.where(test_data["Time_Orderd"].str.contains(':60'),
                                           pd.to_datetime(test_data["Time_Orderd"].str.replace(':60',':00'), format ="%H:%M",errors ="coerce")+timedelta(hours=1),
                                           np.where(test_data["Time_Orderd"].str.contains('24:'),
                                                    pd.to_datetime(test_data["Time_Orderd"].str.replace('24:','00:'), format ="%H:%M",errors ="coerce"),
                                                    pd.to_datetime(test_data["Time_Orderd"], format ="%H:%M",errors ="coerce")))

test_data["timediff"]=test_data["Time_Order_picked"]-test_data["Time_Orderd"]
test_data=NanValue(test_data,["Road_traffic_density","Weather conditions","City"],"timediff","mode")
test_data['timediff'] = test_data['timediff'].fillna(test_data['timediff'].mode()[0])
test_data["Time_Orderd"]=test_data["Time_Orderd"].fillna(test_data["Time_Order_picked"]-test_data['timediff'])

test_data=NanValue(test_data,["Time_Orderd","City"],"Road_traffic_density","mode")
test_data['Road_traffic_density'] = test_data['Road_traffic_density'].fillna(test_data['Road_traffic_density'].mode()[0])
test_data=NanValue(test_data,["Road_traffic_density","City","Order_Date"],"Festival","mode")
test_data=NanValue(test_data,["Order_Date","Time_Order_picked"],"multiple_deliveries","mode")
test_data['multiple_deliveries'] = test_data['multiple_deliveries'].fillna(test_data['multiple_deliveries'].mode()[0])

test_data=NanValue(test_data,["Delivery_person_ID"],"multiple_deliveries","mode")

test_data["Order_Year"]=test_data["Order_Date"].dt.year
test_data["Order_Month"]=test_data["Order_Date"].dt.month
test_data["Order_Day"]=test_data["Order_Date"].dt.day

test_data["Time_Orderd_hr"]=test_data["Time_Orderd"].dt.hour
test_data["Time_Orderd_minute"]=test_data["Time_Orderd"].dt.minute

test_data["Time_Order_picked_hr"]=test_data["Time_Order_picked"].dt.hour
test_data["Time_Order_picked_minute"]=test_data["Time_Order_picked"].dt.minute

test_data.drop(["Order_Date","Time_Orderd","Time_Order_picked"],axis=1,inplace=True)

for i in catergoryData:
    test_data = categoricalDataEndcoding(test_data,train_data_base,i)


X_Testt = test_data[['Delivery_person_Age',
       'Delivery_person_Ratings', 'Restaurant_latitude',
       'Restaurant_longitude', 'Delivery_location_latitude',
       'Delivery_location_longitude', 'Weather conditions',
       'Road_traffic_density', 'Vehicle_condition', 'Type_of_order',
       'Type_of_vehicle', 'multiple_deliveries', 'Festival', 'City','Order_Month',
       'Order_Day', 'Time_Orderd_hr', 'Time_Orderd_minute',
       'Time_Order_picked_hr', 'Time_Order_picked_minute']].values

C:\Users\alish\AppData\Local\Temp\ipykernel_132\1843462781.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [37]:
y_Testt = regressor.predict(X_Testt)

In [39]:
df_submission = pd.DataFrame(y_Testt, columns = ['Time_taken (min)'])

In [43]:
df_final_submission=pd.concat([test_data[["ID"]], df_submission], axis=1, join='inner')

In [46]:
df_final_submission.to_csv("Final_sample_submission.csv",index=False)